In [2]:
import os
from dotenv import load_dotenv
from agents import Agent, Runner, trace, Tool
from agents.mcp import MCPServerStdio
from IPython.display import Markdown, display
from datetime import datetime
from accounts_mcp_client import read_accounts_resource, read_strategy_resource
from accounts import Account

load_dotenv(override=True)

True

### Let's start by gathering the MCP params for our trader

In [3]:
polygon_api_key = os.getenv("POLYGON_API_KEY")
polygon_plan = os.getenv("POLYGON_PLAN")

is_paid_polygon = polygon_plan == "paid"
is_realtime_polygon = polygon_plan == "realtime"

print(is_paid_polygon)
print(is_realtime_polygon)

False
False


In [4]:
if is_paid_polygon or is_realtime_polygon:
    market_mcp = {"command": "uvx","args": ["--from", "git+https://github.com/polygon-io/mcp_polygon@v0.1.0", "mcp_polygon"], "env": {"POLYGON_API_KEY": polygon_api_key}}
else:
    market_mcp = {"command": "uv", "args": ["run", "market_server.py"]}

trader_mcp_server_params = [
    {"command": "uv", "args": ["run", "accounts_server.py"]},
    {"command": "uv", "args": ["run", "push_server.py"]},
    market_mcp
]

### And now for our researcher

In [5]:
researcher_mcp_server_params = [
    {"command": "uv", "args": ["run", "fetch_server.py"]},
    {"command": "uv", "args": ["run", "search_server.py"]},
    {"command": "uv", "args": ["run", "memory_server.py"]}
]

### Now create the MCPServerStdio for each

In [6]:
researcher_mcp_servers = [MCPServerStdio(params, client_session_timeout_seconds=30) for params in researcher_mcp_server_params]
trader_mcp_servers = [MCPServerStdio(params, client_session_timeout_seconds=30) for params in trader_mcp_server_params]
mcp_servers = trader_mcp_servers + researcher_mcp_servers

### Now let's make a Researcher Agent to do market research

And turn it into a tool - remember how this works for OpenAI Agents SDK, and the difference with handoffs?

In [7]:
async def get_researcher(mcp_servers) -> Agent:
    instructions = f"""You are a financial researcher. You are able to search the web for interesting financial news,
look for possible trading opportunities, and help with research.
Based on the request, you carry out necessary research and respond with your findings.
Take time to make multiple searches to get a comprehensive overview, and then summarize your findings.
If there isn't a specific request, then just respond with investment opportunities based on searching latest news.
The current datetime is {datetime.now().strftime("%Y-%m-%d %H:%M:%S")}
"""
    researcher = Agent(
        name="Researcher",
        instructions=instructions,
        model="gpt-4o-mini",
        mcp_servers=mcp_servers,
    )
    return researcher

In [8]:
async def get_researcher_tool(mcp_servers) -> Tool:
    researcher = await get_researcher(mcp_servers)
    return researcher.as_tool(
            tool_name="Researcher",
            tool_description="This tool researches online for news and opportunities, \
                either based on your specific request to look into a certain stock, \
                or generally for notable financial news and opportunities. \
                Describe what kind of research you're looking for."
        )

In [9]:
research_question = "What's the latest news on Amazon?"

for server in researcher_mcp_servers:
    await server.connect()
researcher = await get_researcher(researcher_mcp_servers)
with trace("Researcher"):
    result = await Runner.run(researcher, research_question, max_turns=30)
display(Markdown(result.final_output))



I encountered multiple issues with fetching recent news about Amazon, including errors and invalid links. However, I can summarize common financial topics related to Amazon from recent months. Here's a general overview of what has been happening:

1. **Earnings Reports**: Amazon has been experiencing fluctuations in quarterly earnings, with some analysts optimistic about its cloud services division, Amazon Web Services (AWS). Investors are closely watching performance indicators due to economic uncertainties affecting consumer spending.

2. **Expansion in AI**: Amazon has been investing in artificial intelligence technologies, particularly in enhancing its logistics and retail platforms. This is seen as a major growth area as companies shift towards more automated systems.

3. **Retail Challenges**: The retail landscape continues to pose challenges for Amazon, with increasing competition from other e-commerce and brick-and-mortar retailers. Supply chain issues also remain pertinent as the company seeks to improve efficiency.

4. **Environmental Initiatives**: Amazon is ramping up its sustainability efforts, aiming to achieve net-zero carbon emissions by 2040, which has garnered positive attention from investors focused on environmental, social, and governance (ESG) criteria.

5. **Stock Performance**: Stock price volatility remains a point of interest as analysts debate Amazon's valuation against its growth prospects, especially in light of macroeconomic pressures.

For the most current updates, checking dedicated financial news websites or Amazon's investor relations page directly would provide real-time insights. If you have any specific areas of interest regarding Amazon, or if you want me to focus on a particular topic, please let me know!

### Look at the trace

https://platform.openai.com/traces

In [10]:
ed_initial_strategy = "You are a day trader that aggressively buys and sells shares based on news and market conditions."
Account.get("Onur").reset(ed_initial_strategy)

display(Markdown(await read_accounts_resource("Onur")))
display(Markdown(await read_strategy_resource("Onur")))

{"name": "onur", "balance": 10000.0, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {}, "transactions": [], "portfolio_value_time_series": [["2025-10-08 15:04:19", 10000.0]], "total_portfolio_value": 10000.0, "total_profit_loss": 0.0}

You are a day trader that aggressively buys and sells shares based on news and market conditions.

### And now - to create our Trader Agent

In [11]:
agent_name = "Onur"

account_details = await read_accounts_resource(agent_name)
strategy = await read_strategy_resource(agent_name)

instructions = f"""
You are a trader that manages a portfolio of shares. Your name is {agent_name} and your account is under your name, {agent_name}.
You have access to tools that allow you to search the internet for company news, check stock prices, and buy and sell shares.
Your investment strategy for your portfolio is:
{strategy}
Your current holdings and balance is:
{account_details}
You have the tools to perform a websearch for relevant news and information.
You have tools to check stock prices.
You have tools to buy and sell shares.
You have tools to save memory of companies, research and thinking so far.
Please make use of these tools to manage your portfolio. Carry out trades as you see fit; do not wait for instructions or ask for confirmation.
"""

prompt = """
Use your tools to make decisions about your portfolio.
Investigate the news and the market, make your decision, make the trades, and respond with a summary of your actions.
"""

In [12]:
print(instructions)


You are a trader that manages a portfolio of shares. Your name is Onur and your account is under your name, Onur.
You have access to tools that allow you to search the internet for company news, check stock prices, and buy and sell shares.
Your investment strategy for your portfolio is:
You are a day trader that aggressively buys and sells shares based on news and market conditions.
Your current holdings and balance is:
{"name": "onur", "balance": 10000.0, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {}, "transactions": [], "portfolio_value_time_series": [["2025-10-08 15:04:19", 10000.0], ["2025-10-08 15:04:20", 10000.0]], "total_portfolio_value": 10000.0, "total_profit_loss": 0.0}
You have the tools to perform a websearch for relevant news and information.
You have tools to check stock prices.
You have tools to buy and sell shares.
You have tools to save memory of companies, research and thinking so far.


### And to run our Trader

In [13]:
for server in mcp_servers:
    await server.connect()

researcher_tool = await get_researcher_tool(researcher_mcp_servers)
trader = Agent(
    name=agent_name,
    instructions=instructions,
    tools=[researcher_tool],
    mcp_servers=trader_mcp_servers,
    model="gpt-4o-mini",
)
with trace(agent_name):
    result = await Runner.run(trader, prompt, max_turns=60)
display(Markdown(result.final_output))

### Summary of Actions Taken

1. **Market Research**: I investigated the latest market trends and conducted a price check on major tech stocks: Apple, Amazon, Google, Microsoft, and Tesla.

2. **Stock Purchases**:
   - **Apple (AAPL)**: Bought **100 shares** at a price of **$41.082** per share.
     - **Rationale**: Positive outlook on Apple's service sector and recent product launches.
   - **Amazon (AMZN)**: Bought **50 shares** at a price of **$87.174** per share.
     - **Rationale**: Strong growth expected from Amazon's logistics and cloud services.
   - **Google (GOOGL)**: I attempted to purchase **75 shares**, but there were insufficient funds.
   - **Microsoft (MSFT)**: I attempted to purchase **50 shares**, but there were insufficient funds.
   
3. **Financial Summary**:
   - **Remaining Balance**: $1,533.10
   - **Current Holdings**:
     - **AAPL**: 100 shares
     - **AMZN**: 50 shares
   - **Total Portfolio Value**: $8,733.10
   - **Total Profit/Loss**: -$4,583.80

### Future Steps
- Monitor market conditions for opportunities to sell the current holdings or purchase additional stocks once sufficient funds are available.
- Keep an eye on upcoming earnings reports and any significant news related to current holdings.

### Then go and look at the trace

http://platform.openai.com/traces


In [14]:
await read_accounts_resource(agent_name)

'{"name": "onur", "balance": 1533.0999999999995, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {"AAPL": 100, "AMZN": 50}, "transactions": [{"symbol": "AAPL", "quantity": 100, "price": 41.082, "timestamp": "2025-10-08 15:05:17", "rationale": "Positive outlook on Apple\'s service sector and recent product launches."}, {"symbol": "AMZN", "quantity": 50, "price": 87.174, "timestamp": "2025-10-08 15:05:17", "rationale": "Strong growth expected from Amazon\'s logistics and cloud services."}], "portfolio_value_time_series": [["2025-10-08 15:04:19", 10000.0], ["2025-10-08 15:04:20", 10000.0], ["2025-10-08 15:05:17", 7291.8], ["2025-10-08 15:05:17", 8733.099999999999], ["2025-10-08 15:05:27", 5033.099999999999]], "total_portfolio_value": 5033.099999999999, "total_profit_loss": -9383.800000000003}'

### Now it's time to review the Python module made from this:

`mcp_params.py` is where the MCP servers are specified. You'll notice I've brought in some familiar friends: memory and push notifications!

`templates.py` is where the instructions and messages are set up (i.e. the System prompts and User prompts)

`traders.py` brings it all together.

You'll notice I've done something a bit fancy with code like this:

```
async with AsyncExitStack() as stack:
    mcp_servers = [await stack.enter_async_context(MCPServerStdio(params)) for params in mcp_server_params]
```

This is just a tidy way to combine our "with" statements (known as context managers) so that we don't need to do something ugly like this:

```
async with MCPServerStdio(params=params1) as mcp_server1:
    async with MCPServerStdio(params=params2) as mcp_server2:
        async with MCPServerStdio(params=params3) as mcp_server3:
            mcp_servers = [mcp_server1, mcp_server2, mcp_server3]
```

But it's equivalent.


In [15]:
from traders import Trader

In [16]:
trader = Trader("Onur")

In [17]:
await trader.run()

In [18]:
await read_accounts_resource("Onur")

'{"name": "onur", "balance": 646.3299999999995, "strategy": "You are a day trader that aggressively buys and sells shares based on news and market conditions.", "holdings": {"AAPL": 100, "AMZN": 50, "TSLA": 20, "NFLX": 15, "NVDA": 10}, "transactions": [{"symbol": "AAPL", "quantity": 100, "price": 41.082, "timestamp": "2025-10-08 15:05:17", "rationale": "Positive outlook on Apple\'s service sector and recent product launches."}, {"symbol": "AMZN", "quantity": 50, "price": 87.174, "timestamp": "2025-10-08 15:05:17", "rationale": "Strong growth expected from Amazon\'s logistics and cloud services."}, {"symbol": "TSLA", "quantity": 20, "price": 11.022, "timestamp": "2025-10-08 15:06:34", "rationale": "Bullish on Tesla\'s growth potential post-earnings and increased demand for EVs."}, {"symbol": "NFLX", "quantity": 15, "price": 43.086, "timestamp": "2025-10-08 15:06:34", "rationale": "Positive reception to new content and subscriber growth expectations driven by recent reports."}, {"symbol"


https://platform.openai.com/traces

In [19]:
from mcp_params import trader_mcp_server_params, researcher_mcp_server_params

all_params = trader_mcp_server_params + researcher_mcp_server_params("Onur")

all_tools = [] 
count = 0

for i, each_params in enumerate(all_params, start=1):
    async with MCPServerStdio(params=each_params, client_session_timeout_seconds=60) as server:
        mcp_tools = await server.list_tools()
        tool_names = [tool.name for tool in mcp_tools]  
        all_tools.extend(tool_names)
        print(f"\n--- MCP Server {i}: {each_params['command']} ---")
        for name in tool_names:
            print(f"  • {name}")
        count += len(mcp_tools)

print(f"\nWe have {len(all_params)} MCP servers and {count} tools in total.\n")
print("All tools list:")
print(all_tools)


--- MCP Server 1: uv ---
  • get_balance
  • get_holdings
  • buy_shares
  • sell_shares
  • change_strategy

--- MCP Server 2: uv ---
  • send_notification

--- MCP Server 3: uv ---
  • lookup_share_price

--- MCP Server 4: uv ---
  • fetch_text
  • fetch_json

--- MCP Server 5: uv ---
  • web_search

--- MCP Server 6: uv ---
  • memory_put
  • memory_get
  • memory_list

We have 6 MCP servers and 13 tools in total.

All tools list:
['get_balance', 'get_holdings', 'buy_shares', 'sell_shares', 'change_strategy', 'send_notification', 'lookup_share_price', 'fetch_text', 'fetch_json', 'web_search', 'memory_put', 'memory_get', 'memory_list']
